# 08 - Vertex AI > Notebooks > Models Built in Notebooks with R

# <IN ACTIVE DEVELOPMENT - NOT COMPLETE>

Dev Notes:
- R Kernel:
    - use notebook instance with R kernel
    - or, install R here with needed libraries
- Build R model in notebook environment

Workflow:
- install libraries
- load libraries
- define parameters
- load data: bigrquery or DBI
- Train Model: glm for concept
- Evaluate Model: predictions and confusion matrix for short


Next Steps:
- add:
    - steps to save model local model
    - Vertex AI Model Registry: move to GCS, register in Vertex AI with version
    - serving setup on Vertex
    - add experiment tracking
- package code as script to use in Vertex AI Training Custom Job
- use [reticulate](https://rstudio.github.io/reticulate/) for an R centric workflow

---
## Setup

inputs:

In [10]:
project_id <- system('gcloud config get-value project', intern = TRUE)
project_id

[1] "statmike-mlops-349915"

In [11]:
region <- 'us-central1'
experiment <- '08'
series <- '08'

bq_project <- project_id
bq_dataset <- 'fraud'
bq_table <- 'fraud_prepped'

var_target <- 'Class'
var_omit <- 'transaction_id'

packages:

In [39]:
library(bigrquery)
library(DBI)
library(dplyr)

---

## Data Source

In [75]:
query = sprintf('SELECT * EXCEPT(%s, splits) FROM `%s.%s.%s` WHERE splits = "TRAIN"', var_omit, bq_project, bq_dataset, bq_table)
query

[1] "SELECT * EXCEPT(transaction_id, splits) FROM `statmike-mlops-349915.fraud.fraud_prepped` WHERE splits = \"TRAIN\""

### Retrieve with `bigrquery`

In [76]:
table <- bq_project_query(bq_project, query)
ds1 <- bq_table_download(table, n_max = 2)

In [77]:
dim(ds1)

[1]  2 31

In [82]:
ds1

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
2812,-0.633403,0.9636160,2.4949456,2.0990510,-0.4043307,0.2358616,-0.007931905,0.2114415,-0.2098168,⋯,0.01467553,0.01627818,-0.06146247,0.35519634,-0.1790855,-0.1069474,-0.21503926,0.05069780,0,0
3150,1.313281,-0.2579228,0.1184628,-0.7355566,-0.5693077,-0.7335772,-0.138659177,-0.1416413,1.7080192,⋯,-0.08246737,0.12606591,-0.22315725,-0.07497714,0.9219396,-0.5282835,0.06447566,0.01313189,0,0


### Retrieve with `DBI`

In [79]:
connection <- dbConnect(
    bigrquery::bigquery(),
    project = bq_project,
    dataset = bq_dataset
)

In [80]:
ds2 <- dbGetQuery(connection, query, n = 2)

In [83]:
dim(ds2)

[1]  2 31

In [84]:
ds2

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
2812,-0.633403,0.9636160,2.4949456,2.0990510,-0.4043307,0.2358616,-0.007931905,0.2114415,-0.2098168,⋯,0.01467553,0.01627818,-0.06146247,0.35519634,-0.1790855,-0.1069474,-0.21503926,0.05069780,0,0
3150,1.313281,-0.2579228,0.1184628,-0.7355566,-0.5693077,-0.7335772,-0.138659177,-0.1416413,1.7080192,⋯,-0.08246737,0.12606591,-0.22315725,-0.07497714,0.9219396,-0.5282835,0.06447566,0.01313189,0,0


## Train Model
Using `glm` to fit logistic regression:

In [85]:
query = sprintf('SELECT * EXCEPT(%s, splits) FROM `%s.%s.%s` WHERE splits = "TRAIN"', var_omit, bq_project, bq_dataset, bq_table)
table <- bq_project_query(bq_project, query)
train <- bq_table_download(table)

In [86]:
dim(train)

[1] 228061     31

In [87]:
model <- glm(
    Class ~ .,
    data = train,
    family = binomial)

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [88]:
summary(model)


Call:
glm(formula = Class ~ ., family = binomial, data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.9067  -0.0297  -0.0196  -0.0126   4.6681  

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept) -8.364e+00  2.708e-01 -30.893  < 2e-16 ***
Time        -3.794e-06  2.479e-06  -1.530 0.125941    
V1           1.244e-01  4.665e-02   2.666 0.007667 ** 
V2          -4.491e-02  6.020e-02  -0.746 0.455685    
V3          -1.534e-02  5.767e-02  -0.266 0.790218    
V4           6.668e-01  7.603e-02   8.770  < 2e-16 ***
V5           1.355e-01  6.869e-02   1.973 0.048496 *  
V6          -8.460e-02  7.614e-02  -1.111 0.266556    
V7          -1.222e-01  6.931e-02  -1.763 0.077897 .  
V8          -1.758e-01  3.225e-02  -5.452 4.98e-08 ***
V9          -4.114e-01  1.162e-01  -3.541 0.000398 ***
V10         -8.457e-01  9.909e-02  -8.535  < 2e-16 ***
V11         -1.359e-01  9.153e-02  -1.485 0.137524    
V12          8.785e-02  9.681e-02   

## Evaluate Model

### Retrieve Test Data

In [89]:
query = sprintf('SELECT * EXCEPT(%s, splits) FROM `%s.%s.%s` WHERE splits = "TEST"', var_omit, bq_project, bq_dataset, bq_table)
table <- bq_project_query(bq_project, query)
test <- bq_table_download(table)

In [128]:
dim(test)

[1] 28502    31

### Get Predictions

In [146]:
preds <- predict(model, test, type = "response")

In [147]:
preds[28502]

28502 
0.0002845284

In [148]:
test[28502,'Class']

Class
<int>
0


### Compare Predictions to Actual (Confusion Matrix)

In [149]:
actual <- test[, 'Class']
names(actual) <- 'actual'

In [150]:
actual[1:5,]

actual
<int>
0
0
0
0
0


In [151]:
results <- cbind(actual, tibble(round(preds)))

In [152]:
results[1:5,]

,actual,round(preds)
,<int>,<dbl>
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0


In [153]:
table(results)

      round(preds)
actual     0     1
     0 28452     3
     1    14    33